In [269]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [270]:
%autoreload 2
import pandas as pd
import numpy as np
%aimport AD_comparison_tools
%aimport AD_predictor_tools
%aimport ADpred_LambertTFs_helper
import protfasta

## Converting the 6 part ADPred output to one dataframe of predictions
1. Concatenate the 6 dataframes
2. Merge (remove up to 30 overlapping positions in sequence and activity due to the tiling)
3. For each row, find region of sequence with probability >= threshold for >= min positions. 
4. Convert these predictions to one long list.

In [271]:
#Loading in the output from Savio

part_1 = pd.read_csv("../data/6part_adpred_output/add_adpred__tiled_LambertTFs_1of6.csv", index_col = 0)
part_2 = pd.read_csv("../data/6part_adpred_output/add_adpred__tiled_LambertTFs_2of6.csv", index_col = 0)
part_3 = pd.read_csv("../data/6part_adpred_output/add_adpred__tiled_LambertTFs_3of6.csv", index_col = 0)
part_4 = pd.read_csv("../data/6part_adpred_output/add_adpred__tiled_LambertTFs_4of6.csv", index_col = 0)
part_5 = pd.read_csv("../data/6part_adpred_output/add_adpred__tiled_LambertTFs_5of6.csv", index_col = 0)
part_6 = pd.read_csv("../data/6part_adpred_output/add_adpred__tiled_LambertTFs_6of6.csv", index_col = 0)

df_list = [part_1, part_2, part_3, part_4, part_5, part_6]

In [272]:
combined = AD_comparison_tools.df_list_to_df(df_list)
combined

,ProteinWindowSeq,GeneName,StartPosition,EndPosition,position_wise_prob_adpred
0,MEQYTANSNSSTEQIVVQAGQIQQQQQGGVTAVQLQTEAQVASASG...,sp|P23511|NFYA_HUMAN,0,347,"[5.83147863e-04,6.90653396e-04,1.14429137e-03,..."
1,MSNQYQEEGCSERPECKSKSPTLLSSYCIDSILGRRSPCKMRLLGA...,sp|Q96QS3|ARX_HUMAN,0,562,"[3.22450651e-04,4.81179391e-04,2.97807099e-04,..."
2,MDFDERGPCSSNMYLPSCTYYVSGPDFSSLPSFLPQTPSSRPMTYS...,sp|P31270|HXA11_HUMAN,0,313,"[7.61652645e-03,7.73292175e-03,8.92008469e-03,..."
3,MDPAASSCMRSLQPPAPVWGCLRNPHSEGNGASGLPHYPPTPFSFH...,sp|P50221|MEOX1_HUMAN,0,254,"[5.12628409e-04,5.16732573e-04,4.02507110e-04,..."
4,MLDMSEARSQPPCSPSGTASSMSHVEDSDSDAPPSPAGSEGLGRAG...,sp|P57073|SOX8_HUMAN,0,446,"[5.98106708e-04,6.03391556e-04,6.92091067e-04,..."
...,...,...,...,...,...
1787,MHRTTRIKITELNPHLMCALCGGYFIDATTIVECLHSFCKTCIVRY...,sp|P35227|PCGF2_HUMAN,0,344,"[7.76165252e-05,1.25140708e-04,2.20716174e-04,..."
1788,MDSVSFEDVAVAFTQEEWALLDPSQKNLYRDVMQEIFRNLASVGNK...,sp|Q9BS34|ZN670_HUMAN,0,389,"[1.72792654e-02,2.06266567e-02,1.94855668e-02,..."
1789,MDPEDEGVAGVMSVGPPAARLQEPVTFRDVAVDFTQEEWGQLDPTQ...,sp|P17098|ZNF8_HUMAN,0,575,"[3.98079678e-03,3.63365700e-03,4.65154089e-03,..."
1790,METLTSRHEKRALHSQASAISQDREEKIMSQEPLSFKDVAVVFTEE...,sp|Q9UJW7|ZN229_HUMAN,0,825,"[9.74541035e-05,1.07744912e-04,9.95968730e-05,..."


In [308]:
preds = ADpred_LambertTFs_helper.merge_tiled_sequences(combined)
preds

/Applications/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


,GeneName,ProteinSeq,Length,combined position_wise_prob_adpred
0,sp|P23511|NFYA_HUMAN,MEQYTANSNSSTEQIVVQAGQIQQQQQGGVTAVQLQTEAQVASASG...,347.0,"[0.000583147863, 0.000690653396, 0.00114429137..."
1,sp|Q96QS3|ARX_HUMAN,MSNQYQEEGCSERPECKSKSPTLLSSYCIDSILGRRSPCKMRLLGA...,562.0,"[0.000322450651, 0.000481179391, 0.00029780709..."
2,sp|P31270|HXA11_HUMAN,MDFDERGPCSSNMYLPSCTYYVSGPDFSSLPSFLPQTPSSRPMTYS...,313.0,"[0.00761652645, 0.00773292175, 0.00892008469, ..."
3,sp|P50221|MEOX1_HUMAN,MDPAASSCMRSLQPPAPVWGCLRNPHSEGNGASGLPHYPPTPFSFH...,254.0,"[0.000512628409, 0.000516732573, 0.00040250711..."
4,sp|P57073|SOX8_HUMAN,MLDMSEARSQPPCSPSGTASSMSHVEDSDSDAPPSPAGSEGLGRAG...,446.0,"[0.000598106708, 0.000603391556, 0.00069209106..."
...,...,...,...,...
1603,sp|P35227|PCGF2_HUMAN,MHRTTRIKITELNPHLMCALCGGYFIDATTIVECLHSFCKTCIVRY...,344.0,"[7.76165252e-05, 0.000125140708, 0.00022071617..."
1604,sp|Q9BS34|ZN670_HUMAN,MDSVSFEDVAVAFTQEEWALLDPSQKNLYRDVMQEIFRNLASVGNK...,389.0,"[0.0172792654, 0.0206266567, 0.0194855668, 0.1..."
1605,sp|P17098|ZNF8_HUMAN,MDPEDEGVAGVMSVGPPAARLQEPVTFRDVAVDFTQEEWGQLDPTQ...,575.0,"[0.00398079678, 0.003633657, 0.00465154089, 0...."
1606,sp|Q9UJW7|ZN229_HUMAN,METLTSRHEKRALHSQASAISQDREEKIMSQEPLSFKDVAVVFTEE...,825.0,"[9.74541035e-05, 0.000107744912, 9.9596873e-05..."


In [278]:
df_list = []

for i in range(len(preds.index)):
    df_list.append(ADpred_LambertTFs_helper.return_pred_df(preds["combined position_wise_prob_adpred"][i], 
                                  0.8, 
                                  10,
                                  preds["ProteinSeq"][i],
                                  preds["GeneName"][i]))

In [285]:
adpred_LambertTF = AD_comparison_tools.df_list_to_df(df_list)
adpred_LambertTF["uniprotID"] = adpred_LambertTF.apply(lambda row: row['GeneName'].split('|')[1],axis=1)
adpred_LambertTF["Length"] = adpred_LambertTF["End"] - adpred_LambertTF["Start"] + 1
adpred_LambertTF

,GeneName,ProteinRegionSeq,Start,End,uniprotID,Length
0,sp|Q96QS3|ARX_HUMAN,DEEDEDEEEELLEDDEEE,232.0,249.0,Q96QS3,18.0
1,sp|P57073|SOX8_HUMAN,SEVMGTMDAFD,270.0,280.0,P57073,11.0
2,sp|O14628|ZN195_HUMAN,WKCLDLAQQNLYRDVM,17.0,32.0,O14628,16.0
3,sp|P50549|ETV1_HUMAN,ELFQDLSQLQETWLAEAQ,44.0,61.0,P50549,18.0
4,sp|Q14209|E2F2_HUMAN,LISFSPSLDQDDYLWGLEAGEGISDLFDSYDLGDLLIN,399.0,436.0,Q14209,38.0
...,...,...,...,...,...,...
716,sp|Q8NAP8|ZBT8B_HUMAN,LDIVTSDAFSIILDFLYSGKLD,67.0,88.0,Q8NAP8,22.0
717,sp|P48742|LHX1_HUMAN,GDYQSEYYGPG,279.0,289.0,P48742,11.0
718,sp|Q9UI36|DACH1_HUMAN,PFLFPDGLSSI,607.0,617.0,Q9UI36,11.0
719,sp|P17098|ZNF8_HUMAN,QLDPTQRILYRD,40.0,51.0,P17098,12.0


In [286]:
adpred_LambertTF.to_csv("../output/predictions/adpred_LambertTF_predictions.csv")